In [1]:
# Import relevant libraries

from bs4 import BeautifulSoup as bs
import requests
import smtplib
import time
import datetime

import pandas as pd

from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
import time
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
s = Service(ChromeDriverManager().install())

[WDM] - Downloading: 100%|██████████| 6.68M/6.68M [00:00<00:00, 25.9MB/s]


In [2]:
# Scrape airport data from Wikipedia with Beatiful Soup

url = "https://en.wikipedia.org/wiki/List_of_airports_in_the_United_States#Primary_airports"
table_class = "wikitable sortable jquery-tablesorter"
response = requests.get(url)

soup = bs(response.text, 'html.parser')
airporttable = soup.find('table',{'class':"wikitable"})

In [3]:
# Transferring HTML into a Pandas dataframe

airports = pd.read_html(str(airporttable))
airports = pd.DataFrame(airports[0])

# Clean data to only keep relevant information

del airports['IATA']
del airports['ICAO']
del airports['Airport']

# Keep only large airports (Enplanements > 5,000,000)

#largeairports = airports.loc[(airports.Enplanements > 10000000)]
#largeairports = airports.drop(airports[airports.Enplanements < 10000000].index)
#largeairports.dropna()

# Check for nans

#nan_index_new = largeairports.isna()
#print(nan_index_new.sum())
#print('No nans found!')

In [4]:
# Scrape data from Kayak.com with Selenium

driver = webdriver.Chrome(service=s)


driver.maximize_window()
driver.get("https://www.kayak.com/")
destination = driver.find_element(By.XPATH, "//input[@placeholder='To?']")
destination.send_keys("RNO")

search = driver.find_element(By.CLASS_NAME, "c8LPF-icon").click()
#WebDriverWait(driver, 20).until(EC.element_to_be_clickable((By.CLASS_NAME, "moreButton"))).click()
#time.sleep (5)
#loadmore = driver.find_element(By.XPATH, "//a[@id='c46gA-loadMore']").click()

time.sleep (10)
#testcost = WebDriverWait(driver, 20).until(EC.visibility_of_element_located((By.CLASS_NAME, "price-text")))
not1stop = driver.find_element(By.ID, "LG5S-stops-9789-1-check-icon").click()
not2stops = driver.find_element(By.ID, "LG5S-stops-527-2-check-icon").click()
time.sleep (5)
incoming = driver.find_element(By.ID, "sR_k-value").click()


testcost = driver.find_elements(By.CLASS_NAME, "price-text")
testflightdata = driver.find_elements(By.CLASS_NAME, "top")
testflightdata2 = driver.find_elements(By.CLASS_NAME, "bottom")

dataff = []
totime =[]
fromtime = []
tostop = []
fromstop = []

costclean = []
for values in testcost:
    costclean.append(values.text)
del costclean[1:6]    

for values in testflightdata:
    dataff.append(values.text)
totime = dataff[3:127:8]    
fromtime = dataff[7:127:8]
tostop = dataff[2:127:8]
fromstop = dataff[6:127:8]

airportdest = []
totimeclean = []
for values in totime:
    hrs = int(values[0:1])*60
    minu = int(values[3:5])
    totimeclean.append(hrs + minu)
    airportdest = values

fromtimeclean = []
for values in fromtime:
    hrs = int(values[0:1])*60
    minu = int(values[3:5])
    fromtimeclean.append(hrs + minu)
    
tostopclean = []
for values in tostop:
    if values == "nonstop":
        tostopclean.append(0)
    else:
        tostopclean.append(values[0])
del tostopclean[1]
        
fromstopclean = []
for values in fromstop:
    if values == "nonstop":
        fromstopclean.append(0)
    else:
        fromstopclean.append(values[0])
del fromstopclean[1]

flightdatatest = []
for values in testflightdata2:
    flightdatatest.append(values.text)
toairline = flightdatatest[0:95:6]
fromairline = flightdatatest[3:95:6]
del toairline[1]
del fromairline[1]

cols = ['flightdata']      
#cols = ['cost','flightdata','flightdata2']
dataff = pd.DataFrame(toairline, columns=cols)
print(dataff)
driver.quit()

          flightdata
0  American Airlines
1  American Airlines
2  American Airlines


In [5]:
airports[airports.Enplanements > 10000000]

,City,FAA,Role,Enplanements
41,Phoenix,PHX,P-L,21662580.0
57,Los Angeles,LAX,P-L,42624050.0
66,San Diego,SAN,P-L,12174224.0
67,San Francisco,SFO,P-L,27790717.0
77,Denver,DEN,P-L,31362941.0
90,Fort Lauderdale,FLL,P-L,17612331.0
97,Miami,MIA,P-L,21021640.0
98,Orlando,MCO,P-L,23202480.0
106,Tampa,TPA,P-L,10378514.0
111,Atlanta,ATL,P-L,51865797.0


In [6]:
airportdest = []
costdata = []
flightdata = []
flightdata2 = []

for airport in airports[airports.Enplanements > 10000000]['FAA']:
    driver = webdriver.Chrome(service=s)
#    driver.maximize_window()
    driver.get("https://www.kayak.com/")
    destination = driver.find_element(By.XPATH, "//input[@placeholder='To?']")
    destination.send_keys(airport)

    search = driver.find_element(By.CLASS_NAME, "c8LPF-icon").click()
    #WebDriverWait(driver, 20).until(EC.element_to_be_clickable((By.CLASS_NAME, "moreButton"))).click()
    #time.sleep (5)
    #loadmore = driver.find_element(By.XPATH, "//a[@id='c46gA-loadMore']").click()

    time.sleep (10)

    cost = driver.find_elements(By.CLASS_NAME, "price-text")
    flights = driver.find_elements(By.CLASS_NAME, "top")
    flights2 = driver.find_elements(By.CLASS_NAME, "bottom")
    
    for values in cost:
        costresult = values.text
        costdata.append(costresult)
    for values in flights:
        flightdataresult = values.text
        flightdata.append(flightdataresult)
    for values in flights2:
        flightdata2result = values.text
        flightdata2.append(flightdata2result)
    for values in cost:
        airportdest = [airport]*len(costdata)
    
    print(len(airportdest),len(costdata),len(flightdata),len(flightdata2))
    
    d = {'airport':airportdest,'cost':costdata,'flightdata':flightdata,'flightdata2':flightdata2}
    df = pd.DataFrame(data=d)
    
    
    driver.quit()

9 9 72 54


ValueError: All arrays must be of the same length

In [ ]:
costvalue